In [1]:
from keras.layers import *
from keras.layers import LeakyReLU, UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import time
import pandas as pd

In [2]:
df = pd.read_csv('archive 3\sign_mnist_train.csv')

In [3]:
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27450,13,189,189,190,190,192,193,193,193,193,...,132,165,99,77,52,200,234,200,222,225
27451,23,151,154,157,158,160,161,163,164,166,...,198,198,198,198,198,196,195,195,195,194
27452,18,174,174,174,174,174,175,175,174,173,...,121,196,209,208,206,204,203,202,200,200
27453,17,177,181,184,185,187,189,190,191,191,...,119,56,27,58,102,79,47,64,87,93


In [4]:
generator = 0
discriminator = 0
images = 0

In [5]:
new_df = {}

for label, label_df in df.groupby('label'):
    images = []
    for index, row in tqdm(label_df.iterrows(), total=len(label_df)):
        pixels = row.drop('label')
        images.append(pixels.to_numpy().reshape((28,28, 1)))

    new_df[label] = np.array(images) / 255

100%|██████████| 1118/1118 [00:00<00:00, 1688.37it/s]


In [6]:
GENERATE_RES = 1 # Generation resolution factor 
# (1=32, 2=64, 3=96, 4=128, etc.)
GENERATE_SQUARE = 28 # rows/cols (should be square)
IMAGE_CHANNELS = 1

# Preview image 
PREVIEW_ROWS = 4
PREVIEW_COLS = 7
PREVIEW_MARGIN = 5

# Size vector to generate images from
SEED_SIZE = 100

BATCH_SIZE = 32

print(f"Will generate {GENERATE_SQUARE}px square images.")

Will generate 28px square images.


In [7]:
def build_generator(seed_size):
  model = Sequential()

  model.add(Dense(7 * 7 * 128,activation="relu",input_dim=seed_size))
  model.add(Reshape((7,7, 128)))

  # model.add(UpSampling2D())
  # model.add(Conv2D(256,kernel_size=3,padding="same"))
  # model.add(BatchNormalization(momentum=0.8))
  # model.add(Activation("relu"))

  model.add(UpSampling2D())
  model.add(Conv2D(256,kernel_size=3,padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
   
  model.add(UpSampling2D())
  model.add(Conv2D(128,kernel_size=3,padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))

  model.add(UpSampling2D(size=(GENERATE_RES,GENERATE_RES)))
  model.add(Conv2D(128,kernel_size=3,padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))

  model.add(Conv2D(filters = 1 , kernel_size=3, padding="same"))
  model.add(Activation("tanh"))

  return model

  # input = Input(shape=(seed_size, ))
  # genereated_image = model(input)

  # return Model(input, genereated_image)


def build_discriminator(image_shape):
  model = Sequential()

  model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, 
                     padding="same"))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Dropout(0.25))
  model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Dropout(0.25))
  model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Dropout(0.25))
  model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Dropout(0.25))
  model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))


  return model

  # input_image = Input(shape = image_shape)

  # validaty = model(input_image)

  # return Model(input_image, validaty)

In [8]:
def save_images(cnt,noise, label):
  image_array = np.full(( 
      PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 
      PREVIEW_MARGIN + (PREVIEW_COLS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 1), 
      255, dtype=np.uint8)
  
  generated_images = generator.predict(noise)

  generated_images = 0.5 * generated_images + 0.5

  image_count = 0
  for row in range(PREVIEW_ROWS):
      for col in range(PREVIEW_COLS):
        r = row * (GENERATE_SQUARE+PREVIEW_MARGIN) + PREVIEW_MARGIN
        c = col * (GENERATE_SQUARE+PREVIEW_MARGIN) + PREVIEW_MARGIN
        image_array[r:r+GENERATE_SQUARE,c:c+GENERATE_SQUARE] \
            = generated_images[image_count] * 255
        image_count += 1

          
  output_path = os.path.join('gan_images',f'{label}')
  if not os.path.exists(output_path):
    os.makedirs(output_path)
  
  filename = os.path.join(output_path,f"{cnt}.png")
  plt.imshow(image_array, cmap = 'gray')
  plt.savefig(filename)

In [9]:
7 * 7 * 128

6272

In [10]:
from keras.losses import BinaryCrossentropy
cross_entropy = BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [12]:
@tf.function
def train_step(images):
  seed = tf.random.normal([BATCH_SIZE, SEED_SIZE])

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = generator(seed, training=True)

    real_output = discriminator(images, training=True)
    fake_output = discriminator(generated_images, training=True)

    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)
    

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
  return gen_loss,disc_loss

In [13]:
def hms_string(sec_elapsed):
    """Converts a time in seconds to a string of hours, minutes, and seconds."""
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"

In [25]:
def train(train_images, epochs, batch_size, label):
  fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, 
                                       SEED_SIZE))
  start = time.time()

  for epoch in range(epochs):
    epoch_start = time.time()

    gen_loss_list = []
    disc_loss_list = []

    for i in tqdm(range(0, len(train_images), batch_size)):

      image_batch = train_images[i:i+batch_size]
      @tf.function
      def train_step(images):
        seed = tf.random.normal([BATCH_SIZE, SEED_SIZE])

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
          generated_images = generator(seed, training=True)

          real_output = discriminator(images, training=True)
          fake_output = discriminator(generated_images, training=True)

          gen_loss = generator_loss(fake_output)
          disc_loss = discriminator_loss(real_output, fake_output)
          

          gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
          gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

          generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
          discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
          return gen_loss,disc_loss
          
      t = train_step(image_batch)

      gen_loss_list.append(t[0])
      disc_loss_list.append(t[1])

    g_loss = sum(gen_loss_list) / len(gen_loss_list)
    d_loss = sum(disc_loss_list) / len(disc_loss_list)

    epoch_elapsed = time.time()-epoch_start
    print (f'Epoch {epoch+1}, gen loss={g_loss},disc loss={d_loss},'\
           f' {hms_string(epoch_elapsed)}')
    save_images(epoch,fixed_seed, label)

  elapsed = time.time()-start
  print (f'Training time: {hms_string(elapsed)}')

  generator.save(f"models/{label}_image_generator.h5")
  discriminator.save(f"models/{label}_image_discriminator.h5")

In [26]:
generator_optimizer = Adam(1.5e-4,0.5)
discriminator_optimizer = Adam(1.5e-4,0.5)
image_shape = (GENERATE_SQUARE,GENERATE_SQUARE, 1)

for label in np.sort(df.label.unique())[2:]:
    
    print('training gan for', label)
    images = new_df[label]
    generator = build_generator(SEED_SIZE)
    discriminator = build_discriminator(image_shape)
    train(images, 20, BATCH_SIZE, label)
    

training gan for 2


 11%|█         | 4/36 [00:08<01:06,  2.07s/it]

 14%|█▍        | 5/36 [00:10<01:06,  2.13s/it]

100%|██████████| 36/36 [01:13<00:00,  2.04s/it]


Epoch 1, gen loss=2.6943869590759277,disc loss=0.4680771231651306, 0:01:13.27
1/1 [==============================] - 0s 262ms/step


...

Epoch 19, gen loss=3.0824007987976074,disc loss=0.5456675291061401, 0:05:09.13
1/1 [==============================] - 0s 300ms/step


100%|██████████| 36/36 [04:54<00:00,  8.17s/it]


Epoch 20, gen loss=3.2675468921661377,disc loss=0.49150168895721436, 0:04:54.27
1/1 [==============================] - 0s 247ms/step
Training time: 0:34:31.08
training gan for 3


 66%|██████▌   | 25/38 [03:46<01:57,  9.04s/it]


KeyboardInterrupt: 